In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [3]:
data['Diagnosis'].unique()

array(['normal', 'early neoplasia', 'cancer'], dtype=object)

In [4]:
data['Diagnosis'].value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [5]:
normal_dt = data[data['Diagnosis']=='normal']
en_dt = data[data['Diagnosis']=='early neoplasia']
cancer_dt = data[data['Diagnosis']=='cancer']

In [6]:
normal_data = normal_dt.drop(['Patient_id','Diagnosis'],axis=1)
en_data = en_dt.drop(['Patient_id','Diagnosis'],axis=1)
cancer_data = cancer_dt.drop(['Patient_id','Diagnosis'],axis=1)

In [7]:
normal_data.columns

Index([u'LOC643837', u'LOC100130417', u'SAMD11', u'NOC2L', u'KLHL17',
       u'PLEKHN1', u'C1orf170', u'HES4', u'ISG15', u'AGRN',
       ...
       u'CLIC2', u'RPS4Y1', u'ZFY', u'PRKY', u'USP9Y', u'DDX3Y', u'CD24',
       u'CYorf15B', u'KDM5D', u'EIF1AY'],
      dtype='object', length=15748)

# Задание 1. Найти гены, которые отличаются в разных группах

### H0 - средняя эксперессивность одинаковая
### H1 - разная
### Проверка через критерий стьюдента для независимых измерений

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [8]:
import scipy
from statsmodels.stats.weightstats import *
count_p_more_1 = 0
count_p_less_1 = 0
count_p_more_2 = 0
count_p_less_2 = 0
for col in normal_data.columns:
    stat, p = scipy.stats.ttest_ind(normal_data[col], en_data[col], equal_var = False) #значение статистики и уровень значимости
    if p>0.05:
        count_p_more_1 += 1
    else:
        count_p_less_1 += 1
print 'count_p_more_1=', count_p_more_1
print 'count_p_less_1=', count_p_less_1

for col in en_data.columns:
    stat, p = scipy.stats.ttest_ind(en_data[col], cancer_data[col], equal_var = False) #значение статистики и уровень значимости
    if p>0.05:
        count_p_more_2 += 1
    else:
        count_p_less_2 += 1
print 'count_p_more_2=', count_p_more_2
print 'count_p_less_2=', count_p_less_2
print 'count_p_more_all=', count_p_more_1 + count_p_more_2
print 'count_p_less_all=', count_p_less_1 + count_p_less_2

count_p_more_1= 14173
count_p_less_1= 1575
count_p_more_2= 12258
count_p_less_2= 3490
count_p_more_all= 26431
count_p_less_all= 5065


### Часть2. С учетом поправок Холма на множественную проверку гипотез

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.51.51.5.


In [9]:
p_list_1 = list()
p_list_2 = list()
for col in normal_data.columns:
    stat1, p1 = scipy.stats.ttest_ind(normal_data[col], en_data[col], equal_var = False) #значение статистики и уровень значимости
    p_list_1.append(p1)
    stat2, p2 = scipy.stats.ttest_ind(en_data[col], cancer_data[col], equal_var = False) #значение статистики и уровень значимости
    p_list_2.append(p2)

In [10]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject1, p_corrected_1, a1, a2 = multipletests(p_list_1, alpha = 0.05/2, method = 'holm') 
reject2, p_corrected_2, a1, a2 = multipletests(p_list_2, alpha = 0.05/2, method = 'holm') 

In [39]:
count_p_more_1 = 0
count_p_less_1 = 0
count_p_more_2 = 0
count_p_less_2 = 0
f1_list = list()
f2_list = list()
for i in range(0,len(p_corrected_1)):
    if p_corrected_1[i]<0.025:
        count_p_less_1 += 1
        C = normal_data[normal_data.columns[i]].mean()
        T = en_data[en_data.columns[i]].mean()
        if T > C:
            F_1 = T/float(C)
        else:
            F_1 = -C/float(T)
        f1_list.append(F_1)
    else:
        count_p_more_1 += 1
for i in range(0,len(p_corrected_2)):
    if p_corrected_2[i]<0.025:
        count_p_less_2 += 1
        C = en_data[en_data.columns[i]].mean()
        T = cancer_data[cancer_data.columns[i]].mean()
        if T > C:
            F_2 = T/float(C)
        else:
            F_2 = -C/float(T)
        f2_list.append(F_2)
    else:
        count_p_more_2 += 1
print 'count_p_more_1=', count_p_more_1
print 'count_p_less_1=', count_p_less_1
print 'count_p_more_2=', count_p_more_2
print 'count_p_less_2=', count_p_less_2

count_p_more_1= 15746
count_p_less_1= 2
count_p_more_2= 15669
count_p_less_2= 79


In [40]:
print 'f1:',f1_list
print 'f2:',f2_list

f1: [1.509785482044398, 1.9748676656368698]
f2: [1.8438601120360605, -2.383209744151477, -1.9973807408171325, -1.6934276114377385, -1.795639684357122, -2.119522587228923, -1.6300230825502287, -2.4492326684105192, -3.033566155995153, -2.79378540308933, -2.125933494769897, -1.9377959456382408, -1.6743935503606164, -2.004977539524381, -4.364839773317585, -2.3234996260334766, -1.7472427023789787, -2.045648683740802, -1.6475239475535153, -2.736973729291268, -2.1404583552879264, -1.548831064278174, -2.5764990178903804, -1.8542308025419714, -2.905515073516559, 2.2887923692234415, -2.6759324726496403, -2.2089634178013275, -2.0024024200136856, -2.153608320161442, -1.8164750122406366, -1.71870946563695, -2.1201772631710583, -1.9963596951256357, -2.0933818653389027, -2.8178876086244475, -2.246411485223897, -2.3804962593091274, -1.8175706684521358, -2.784621568610554, -2.442654044605397, -1.8195596254495814, -2.279085957408533, -2.675435193552647, -3.3521459750889795, -1.8933656951331517, -1.99008

In [41]:
f1_list.extend(f2_list)

In [42]:
n_count = 0
for f in f1_list:
    if abs(f)>1.5:
        n_count +=1
print n_count

79


# Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 


In [43]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject1, p_corrected_1, a1, a2 = multipletests(p_list_1, alpha = 0.05/2, method = 'fdr_bh') 
reject2, p_corrected_2, a1, a2 = multipletests(p_list_2, alpha = 0.05/2, method = 'fdr_bh')

In [44]:
count_p_more_1 = 0
count_p_less_1 = 0
count_p_more_2 = 0
count_p_less_2 = 0
f1_list = list()
f2_list = list()
for i in range(0,len(p_corrected_1)):
    if p_corrected_1[i]<0.025:
        count_p_less_1 += 1
        C = normal_data[normal_data.columns[i]].mean()
        T = en_data[en_data.columns[i]].mean()
        if T > C:
            F_1 = T/float(C)
        else:
            F_1 = -C/float(T)
        f1_list.append(F_1)
    else:
        count_p_more_1 += 1
for i in range(0,len(p_corrected_2)):
    if p_corrected_2[i]<0.025:
        count_p_less_2 += 1
        C = en_data[en_data.columns[i]].mean()
        T = cancer_data[cancer_data.columns[i]].mean()
        if T > C:
            F_2 = T/float(C)
        else:
            F_2 = -C/float(T)
        f2_list.append(F_2)
    else:
        count_p_more_2 += 1
print 'count_p_more_1=', count_p_more_1
print 'count_p_less_1=', count_p_less_1
print 'count_p_more_2=', count_p_more_2
print 'count_p_less_2=', count_p_less_2

count_p_more_1= 15744
count_p_less_1= 4
count_p_more_2= 14916
count_p_less_2= 832


In [45]:
f1_list.extend(f2_list)

In [46]:
n_count = 0
for f in f1_list:
    if abs(f)>1.5:
        n_count +=1
print n_count

528
